In [1]:
!pip install biopython
!pip install pysam

Looking in indexes: https://pypi.python.org/simple, https://cognite.jfrog.io/cognite/api/pypi/snakepit/simple
Looking in indexes: https://pypi.python.org/simple, https://cognite.jfrog.io/cognite/api/pypi/snakepit/simple


In [8]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import pairwise2


/home/priyanka/etc/anaconda3/lib/python3.9/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [3]:
def read_fastq(filename):
    sequences = []
    with open(filename, 'r') as file:
        while True:
            header = file.readline().strip()
            if not header:
                break
            sequence = file.readline().strip()
            file.readline()  # Skip the "+"
            quality = file.readline().strip()
            sequences.append((header, sequence, quality))
    return sequences

In [4]:
def find_overlap(read1, read2, min_overlap=10):
    """Find the overlap between two reads"""
    len1 = len(read1)
    len2 = len(read2)
    max_overlap = min(len1, len2)

    for overlap in range(max_overlap, min_overlap - 1, -1):
        if read1.endswith(read2[:overlap]):
            return overlap
    return 0

In [5]:
def merge_reads(read1, read2, overlap):
    """Merge two reads using the given overlap"""
    return read1 + read2[overlap:]

In [6]:
def olc_assembly(r1_reads, r2_reads):
    """Assemble paired-end reads using the OLC algorithm"""
    assembled_sequences = []
    for read1 in r1_reads:
        best_overlap = 0
        best_read2 = None

        for read2 in r2_reads:
            overlap = find_overlap(read1[1], read2[1])
            if overlap > best_overlap:
                best_overlap = overlap
                best_read2 = read2

        if best_read2 is not None:
            assembled_sequence = merge_reads(read1[1], best_read2[1], best_overlap)
            assembled_quality = read1[2] + best_read2[2][best_overlap:]
            assembled_sequences.append((read1[0], assembled_sequence, assembled_quality))

    return assembled_sequences


In [7]:
# Read paired-end reads from FASTQ files
r1_fastq_reads = read_fastq("data/mutant_R1.fastq")
r2_fastq_reads = read_fastq("data/mutant_R2.fastq")

In [8]:
# Perform overlap-layout-consensus assembly
olc_assembled_sequences = olc_assembly(r1_reads=r1_fastq_reads, r2_reads=r2_fastq_reads)
olc_assembled_sequences

KeyboardInterrupt: 

In [ ]:
# Write assembled sequences to a FASTA file
with open("results/olc_assembled_sequences.fasta", "w") as output_handle:
    for olc_header, old_sequence, olc_quality in olc_assembled_sequences:
        output_handle.write(f">{olc_header}\n{old_sequence}\n")

# Comparison

In [9]:
def calculate_n50_length(contig_lengths):
    sorted_lengths = sorted(contig_lengths, reverse=True)
    total_length = sum(sorted_lengths)
    target_length = total_length / 2
    cumulative_length = 0
    for length in sorted_lengths:
        cumulative_length += length
        if cumulative_length >= target_length:
            return length

In [10]:
def calculate_genome_coverage(assembled_sequences, reference_sequences):
    total_reference_length = sum(len(seq) for seq in reference_sequences)
    covered_length = 0
    for assembled_seq in assembled_sequences:
        assembled_seq = assembled_seq.lower()
        print(f"Processing assembled sequence: {assembled_seq}")
        for reference_seq in reference_sequences:
            reference_seq = reference_seq.lower()
            alignments = pairwise2.align.localms(reference_seq, assembled_seq, 2, -1, -0.5, -0.1)
            for alignment in alignments:
                covered_length += (alignment.end - alignment.start)
                break
    return covered_length / total_reference_length

In [11]:
# Read assembled genome
with open("results/olc_assembled_sequences.fasta", "r") as results_file:
    olc_assembled_sequences_fasta = [olc_seq.strip() for olc_seq in results_file.readlines() if not olc_seq.startswith(">")]

print(f"OLC assembled sequences: {len(olc_assembled_sequences_fasta)}")

OLC assembled sequences: 10784


In [12]:
# Read reference genome
with open("data/wildtype.fna", "r") as ref_file:
    reference_sequences_fna = [ref_seq.strip() for ref_seq in ref_file.readlines() if not ref_seq.startswith(">")]

print(f"Reference sequences: {len(reference_sequences_fna)}")

Reference sequences: 3290


In [13]:
# Calculate N50 length
old_contig_lengths = [len(seq) for seq in olc_assembled_sequences_fasta]
print(old_contig_lengths)

[222, 154, 154, 204, 158, 163, 211, 181, 246, 169, 157, 277, 150, 192, 184, 260, 237, 209, 280, 154, 285, 270, 238, 271, 205, 191, 155, 159, 192, 268, 236, 217, 270, 233, 290, 188, 243, 289, 225, 236, 269, 244, 261, 154, 180, 211, 275, 181, 187, 167, 170, 168, 181, 284, 239, 257, 154, 254, 158, 164, 290, 168, 203, 193, 179, 250, 252, 180, 179, 249, 181, 175, 188, 283, 195, 153, 259, 191, 233, 154, 203, 171, 168, 202, 205, 188, 223, 154, 248, 157, 169, 202, 178, 168, 249, 154, 202, 289, 273, 195, 285, 183, 176, 210, 229, 219, 167, 175, 166, 269, 222, 174, 154, 219, 166, 258, 166, 178, 266, 159, 182, 198, 185, 234, 152, 288, 253, 203, 265, 235, 255, 234, 269, 181, 167, 206, 203, 218, 175, 173, 286, 195, 178, 207, 237, 242, 175, 272, 236, 208, 225, 268, 213, 161, 282, 171, 179, 156, 164, 208, 191, 172, 253, 155, 244, 184, 200, 222, 208, 158, 268, 169, 219, 170, 168, 263, 162, 198, 229, 178, 198, 211, 273, 282, 203, 210, 155, 181, 196, 157, 217, 271, 158, 186, 190, 272, 229, 176, 183, 209,

In [14]:
n50_length = calculate_n50_length(old_contig_lengths)
print(n50_length)

213


In [16]:
# Calculate genome coverage
# genome_coverage = calculate_genome_coverage(olc_assembled_sequences_fasta, reference_sequences_fna)
# genome_coverage

Processing assembled sequence: aatgttgtcacttggattcaaatgacattttaaatctaattattcatgaatcgaactagtacgaaatgcaatgagcatcttgtctagttcgattttttaatgtctaaaaatgtcgtatatgtaatcagagtagaaagtgttgaggcgtttcagaagttgtttagaaaagtaagtaaaataaaaaatgcactgagcaacaaaagatgttgctcgtgcatttag


KeyboardInterrupt: 